# Hyperparameter Optimization

In [1]:

from pathlib import Path
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from carde.io import tiled_image_path, tiled_label_path
from carde.dataset import SEMTileDataset
from carde.dataset import split_data
from carde.unet import SegmentationModel
from carde.evalmetrics import evaluate_model

import optuna

devices=(0,)
_ = torch.manual_seed(42)

## Figure parameters

In [2]:
# figure parameters
plt.rcParams["svg.fonttype"] = "none"  # editable text in svg vector formats
plt.rcParams["font.size"] = 8
plt.rcParams["axes.labelsize"] = 8
plt.rcParams["axes.titlesize"] = 10
plt.rcParams["xtick.labelsize"] = 6
plt.rcParams["ytick.labelsize"] = plt.rcParams["xtick.labelsize"]

# figure sizes
textwidth = 7.05  # inches
onecolumn = 3.42  # inches
# set the default figure size to one column width
plt.rcParams["figure.figsize"] = (textwidth, onecolumn)

figure_path = Path("../paper/figures")
figure_path.mkdir(parents=True, exist_ok=True)

## Prepare dataset

In [3]:
dataset = SEMTileDataset(tiled_image_path, tiled_label_path, train=True)
# create dataloader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# split data into three sets: train, validation, and test
train_loader, val_loader, test_loader = split_data(dataset, batch_size=32, train_ratio=0.8, val_ratio=0.1)

In [4]:
print(f"Batches in training data loader: {len(train_loader)}")
print(f"Batches in validation data loader: {len(val_loader)}")
print(f"Batches in test data loader: {len(test_loader)}")

print(f"Number of samples in training data loader: {len(train_loader.dataset)}")
print(f"Number of samples in validation data loader: {len(val_loader.dataset)}")
print(f"Number of samples in test data loader: {len(test_loader.dataset)}")

total_samples = len(train_loader.dataset) + len(val_loader.dataset) + len(test_loader.dataset)
print(f"Total number of samples: {total_samples}")


Batches in training data loader: 48
Batches in validation data loader: 6
Batches in test data loader: 6
Number of samples in training data loader: 1536
Number of samples in validation data loader: 192
Number of samples in test data loader: 192
Total number of samples: 1920


## Define objective function

In [5]:
def objective(trial:optuna.Trial)->float:
    patience = trial.suggest_int("early_stopping",12,20)
    early_stop_callback = EarlyStopping(
        monitor="validation_loss",
        mode="min",
        patience=patience,
        verbose=True
    )
    model = SegmentationModel(
        in_channels=2,
        out_channels=1,
        lr=trial.suggest_float("lr", 0.0002, 0.0002, log=True),
        lr_scheduler_patience=patience // 2,
        blocks=trial.suggest_int("Unet_blocks", 3, 3),
        hidden_layers_block1=2 ** trial.suggest_int("log2_hidden_layers_block1", 6, 10),
        pooling_range=2,
    )
    trainer = pl.Trainer(max_epochs=1000, devices=devices, callbacks=[early_stop_callback])
    trainer.fit(model, train_loader, val_loader)
    avg_loss, avg_score = evaluate_model(model, val_loader, trainer, show_sample=False)
    return avg_loss

## Run optimization

In [6]:
study_name = "optimize_unet_sheduler_3"

study = optuna.create_study(direction="minimize", study_name=study_name,storage=f"sqlite:///{study_name}.db", load_if_exists=True)

[I 2025-08-29 15:50:32,178] Using an existing study with name 'optimize_unet_sheduler_3' instead of creating a new one.


In [15]:
study.optimize(objective, n_trials=10)

/home/korten76/workspace/chekhonin-automatic-carbide-detection/venv/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: PossibleUserWarning:

The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/korten76/workspace/chekhonin-automatic-carbide ...

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/s

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/korten76/workspace/chekhonin-automatic-carbide-detection/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.

/home/korten76/workspace/chekhonin-automatic-carbide-detection/venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.

/home/korten76/workspace/chekhonin-automatic-carbide-detection/venv/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: PossibleUserWarning:

The number of training batches (48) is smaller than the logging interval Train

Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

/home/korten76/workspace/chekhonin-automatic-carbide-detection/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [9]:
study.optimize(objective, n_trials=30)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name  | Type | Params | Mode 
---------------------------------------
0 | model | UNet | 462 K  | train
---------------------------------------
462 K     Trainable params
0         Non-trainable params
462 K     Total params
1.852     Total estimated model params size (MB)
50        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric validation_loss improved. New best score: 0.840


...

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric validation_loss did not improve in the last 12 records. Best score: 0.055. Signaling Trainer to stop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Testing: |          | 0/? [00:00<?, ?it/s]

[I 2025-08-27 03:29:39,083] Trial 39 finished with value: 0.05674080550670624 and parameters: {'early_stopping': 12, 'lr': 0.0009722936469724459, 'Unet_blocks': 2, 'log2_hidden_layers_block1': 6}. Best is trial 21 with value: 0.046181317418813705.


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.05674080550670624
       test_score           0.9432591795921326
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Average Dice Loss: 0.0567
Average 1 - Dice Loss: 0.9433


# Evaluate optimization results

In [7]:
study.best_params

{'early_stopping': 14,
 'lr': 0.0002,
 'Unet_blocks': 3,
 'log2_hidden_layers_block1': 7}

In [8]:
df = study.trials_dataframe()
df

,number,value,datetime_start,datetime_complete,duration,params_Unet_blocks,params_early_stopping,params_log2_hidden_layers_block1,params_lr,state
0,0,0.066554,2025-08-26 23:58:11.911054,2025-08-27 00:01:45.193823,0 days 00:03:33.282769,2,11,6,0.001328,COMPLETE
1,1,0.057634,2025-08-27 00:01:45.222408,2025-08-27 00:05:21.066132,0 days 00:03:35.843724,2,7,6,0.001119,COMPLETE
2,2,0.057148,2025-08-27 00:05:21.094135,2025-08-27 00:07:32.525474,0 days 00:02:11.431339,3,8,4,0.001411,COMPLETE
3,3,0.068220,2025-08-27 00:07:32.552178,2025-08-27 00:09:49.036780,0 days 00:02:16.484602,4,9,4,0.001530,COMPLETE
4,4,0.046612,2025-08-27 00:09:49.066313,2025-08-27 00:16:33.246264,0 days 00:06:44.179951,2,10,6,0.000290,COMPLETE
5,5,0.066099,2025-08-27 00:16:33.284749,2025-08-27 00:19:13.437550,0 days 00:02:40.152801,3,11,4,0.002115,COMPLETE
6,6,0.072145,2025-08-27 00:19:13.467909,2025-08-27 00:20:42.171888,0 days 00:01:28.703979,4,12,3,0.002458,COMPLETE
7,7,0.055292,2025-08-27 00:20:42.199955,2025-08-27 00:23:50.115456,0 days 00:03:07.915501,2,6,3,0.000583,COMPLETE
8,8,0.071400,2025-08-27 00:23:50.142131,2025-08-27 00:25:34.550489,0 days 00:01:44.408358,4,6,3,0.001054,COMPLETE
9,9,0.074888,2025-08-27 00:25:34.582266,2025-08-27 00:27:32.939494,0 days 00:01:58.357228,4,9,3,0.000576,COMPLETE


In [9]:
study.best_trial

FrozenTrial(number=40, state=1, values=[0.04550115391612053], datetime_start=datetime.datetime(2025, 8, 27, 6, 40, 5, 216728), datetime_complete=datetime.datetime(2025, 8, 27, 6, 59, 42, 754087), params={'early_stopping': 14, 'lr': 0.0002, 'Unet_blocks': 3, 'log2_hidden_layers_block1': 7}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'early_stopping': IntDistribution(high=20, log=False, low=12, step=1), 'lr': FloatDistribution(high=0.0002, log=True, low=0.0002, step=None), 'Unet_blocks': IntDistribution(high=3, log=False, low=3, step=1), 'log2_hidden_layers_block1': IntDistribution(high=10, log=False, low=6, step=1)}, trial_id=41, value=None)

In [10]:
df_filtered = df[df["value"] < 0.1].copy()
df_filtered

,number,value,datetime_start,datetime_complete,duration,params_Unet_blocks,params_early_stopping,params_log2_hidden_layers_block1,params_lr,state
0,0,0.066554,2025-08-26 23:58:11.911054,2025-08-27 00:01:45.193823,0 days 00:03:33.282769,2,11,6,0.001328,COMPLETE
1,1,0.057634,2025-08-27 00:01:45.222408,2025-08-27 00:05:21.066132,0 days 00:03:35.843724,2,7,6,0.001119,COMPLETE
2,2,0.057148,2025-08-27 00:05:21.094135,2025-08-27 00:07:32.525474,0 days 00:02:11.431339,3,8,4,0.001411,COMPLETE
3,3,0.068220,2025-08-27 00:07:32.552178,2025-08-27 00:09:49.036780,0 days 00:02:16.484602,4,9,4,0.001530,COMPLETE
4,4,0.046612,2025-08-27 00:09:49.066313,2025-08-27 00:16:33.246264,0 days 00:06:44.179951,2,10,6,0.000290,COMPLETE
5,5,0.066099,2025-08-27 00:16:33.284749,2025-08-27 00:19:13.437550,0 days 00:02:40.152801,3,11,4,0.002115,COMPLETE
6,6,0.072145,2025-08-27 00:19:13.467909,2025-08-27 00:20:42.171888,0 days 00:01:28.703979,4,12,3,0.002458,COMPLETE
7,7,0.055292,2025-08-27 00:20:42.199955,2025-08-27 00:23:50.115456,0 days 00:03:07.915501,2,6,3,0.000583,COMPLETE
8,8,0.071400,2025-08-27 00:23:50.142131,2025-08-27 00:25:34.550489,0 days 00:01:44.408358,4,6,3,0.001054,COMPLETE
9,9,0.074888,2025-08-27 00:25:34.582266,2025-08-27 00:27:32.939494,0 days 00:01:58.357228,4,9,3,0.000576,COMPLETE


In [11]:
optuna.visualization.plot_parallel_coordinate(study)

In [13]:
optuna.visualization.plot_optimization_history(study)

In [14]:
fig = optuna.visualization.plot_slice(study)
fig.update_yaxes(range=[0.04, 0.08])
fig.update_layout(width=800, height=400)
fig